# Тестовое задание. Дорошев.

Вывести отчёт по заявкам за произвольный период в днях (от 1 дня и более)  
Данные нужно забрать с помощью Google Sheets API из документа https://docs.google.com/spreadsheets/d/1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ  
Формат отчёта:  
**Dimensions**
- канал привлечения заявки (d_utm_source)
- клуб (d_club)
- менеджер (d_manager)

**Metrics**
- количество заявок
- количество мусорных заявок (на основании заявки не создан клиент)
- количество новых заявок (не было заявок и покупок от этого клиента раньше) 
- количество покупателей (кто купил в течение недели после заявки) 
- количество новых покупателей (кто купил в течение недели после заявки, и не покупал раньше)
- доход от покупок новых покупателей

Итоговый отчёт нужно вывести в свой документ Google Sheets:
- Данные загружать в него через Google Sheets API
- Для итогового отчёта использовать pivot table Google Sheets
- (Бонусное задание, необязательно) вывести итоговый отчёт в виде pivot table DataStudio

Срок сдачи задания: 4 календарных дня
Если по заданию будут вопросы, то пиши вопросы в Тг: https://t.me/theberk

In [1]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

## Импортируем данные используя API Google Sheets

In [2]:
gc = gspread.service_account(filename='testtask-314915-9d01d59d7961.json')
#sh_key = gc.open_by_url('https://docs.google.com/spreadsheets/d/1ZyafGUVeXxNxcdOlCbxB15LGMLSKd6da56rRCsLLQdA/edit?usp=sharing')
sh = gc.open_by_key('1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ')

### Transactions

In [3]:
worksheet = sh.get_worksheet(1) #-> 0 - first sheet, 1 - second sheet etc. 
list_of_dicts = worksheet.get_all_records()
trans =  pd.DataFrame.from_dict(list_of_dicts)

In [4]:
trans.sample(3)

,transaction_id,created_at,m_real_amount,l_client_id
25471,c4ed258d-c3e2-4292-be59-06496b7f3145,2020-10-07 13:30:49,5262,a916e6d4-5ad1-11ea-abeb-c412f533dba1
12530,05712056-d27b-40fc-9eed-c11ae92b84f9,2020-01-31 15:23:33,61,67e73af2-1c86-11e8-8130-c412f533dba1
4728,2bb44867-e433-4dd3-8ba5-901827745cc3,2020-01-19 13:59:09,2608,d1e5b58a-edfb-11e7-8126-c412f533dba1


In [5]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29000 entries, 0 to 28999
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   transaction_id  29000 non-null  object
 1   created_at      29000 non-null  object
 2   m_real_amount   29000 non-null  int64 
 3   l_client_id     29000 non-null  object
dtypes: int64(1), object(3)
memory usage: 906.4+ KB


In [6]:
trans['created_at'] = pd.to_datetime(trans['created_at'])

In [7]:
trans.duplicated().sum()

13

In [8]:
trans = trans.drop_duplicates()

In [9]:
trans.describe(include = 'all')

<ipython-input-9-47fb0d76c2c1>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  trans.describe(include = 'all')


,transaction_id,created_at,m_real_amount,l_client_id
count,28987,28987,28987.000000,28987
unique,23837,23492,NaN,9342
top,f409b2a7-2a1c-4da4-94af-025dc536f9c6,2020-03-01 11:19:18,NaN,8e0b4320-6fd8-11e7-80fc-c412f533dba1
freq,22,8,NaN,231
first,NaN,2020-01-01 07:59:51,NaN,NaN
last,NaN,2020-11-28 12:02:52,NaN,NaN
mean,NaN,NaN,10656.674061,NaN
std,NaN,NaN,19821.517220,NaN
min,NaN,NaN,-157019.000000,NaN
25%,NaN,NaN,199.000000,NaN


### Clients

In [10]:
worksheet = sh.get_worksheet(2) #-> 0 - first sheet, 1 - second sheet etc. 
cl =  pd.DataFrame.from_dict(worksheet.get_all_records())

In [11]:
cl.sample(3)

,client_id,created_at,l_manager_id
64488,d9ab65aa-d5b3-11e9-abbe-c412f533dba1,0001-01-01 00:00:00,543888c4-fbb3-11e9-abdc-c412f533dba1
48707,a4179c58-d010-11ea-ac0f-c412f533dba1,2020-07-27 17:53:12,d12fe765-b444-11e9-abb3-c412f533dba1
30173,64059ef5-d5a8-11e9-abbe-c412f533dba1,0001-01-01 00:00:00,b832aac3-fcda-11ea-ac18-c412f533dba1


In [12]:
cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75766 entries, 0 to 75765
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   client_id     75766 non-null  object
 1   created_at    75766 non-null  object
 2   l_manager_id  75766 non-null  object
dtypes: object(3)
memory usage: 1.7+ MB


In [13]:
cl.duplicated().sum()

0

In [14]:
cl.describe(include = 'all')

,client_id,created_at,l_manager_id
count,75766,75766,75766
unique,75766,14851,44
top,4e4bd42a-d5b3-11e9-abbe-c412f533dba1,0001-01-01 00:00:00,9a87c608-52dc-11ea-abeb-c412f533dba1
freq,1,29215,11373


### Managers

In [15]:
worksheet = sh.get_worksheet(3) #-> 0 - first sheet, 1 - second sheet etc. 
man =  pd.DataFrame.from_dict(worksheet.get_all_records())

In [16]:
man

,manager_id,d_manager,d_club
0,1e9c5562-0cfc-11ea-abe1-c412f533dba1,manager #8,club #2
1,43756fa4-57a0-11e9-ab9a-c412f533dba1,manager #11,club #4
2,543888c4-fbb3-11e9-abdc-c412f533dba1,manager #12,club #3
3,5dd88f32-6501-11ea-abf3-c412f533dba1,manager #10,club #2
4,9a87c608-52dc-11ea-abeb-c412f533dba1,manager #9,club #3
5,abed5496-ca88-11e9-abb8-c412f533dba1,manager #5,club #4
6,ad52c7a8-a752-11e7-8115-c412f533dba1,manager #7,club #1
7,b55a233c-5df6-11ea-abf2-c412f533dba1,manager #2,club #4
8,b832aac3-fcda-11ea-ac18-c412f533dba1,manager #13,club #3
9,d12fe765-b444-11e9-abb3-c412f533dba1,manager #1,club #1


In [17]:
man.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   manager_id  13 non-null     object
 1   d_manager   13 non-null     object
 2   d_club      13 non-null     object
dtypes: object(3)
memory usage: 440.0+ bytes


In [18]:
man.duplicated().sum()

0

In [19]:
man.describe(include = 'all')

,manager_id,d_manager,d_club
count,13,13,13
unique,13,13,4
top,5dd88f32-6501-11ea-abf3-c412f533dba1,manager #10,club #2
freq,1,1,4


### Leads

In [20]:
worksheet = sh.get_worksheet(4) #-> 0 - first sheet, 1 - second sheet etc. 
leads =  pd.DataFrame.from_dict(worksheet.get_all_records())

In [21]:
leads.sample(10)

,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id
442,1fdfa8c1-1df7-11eb-ac1d-c412f533dba1,2020-11-03 21:08:00,cpc,instagram,db06b8bc-bfb2-11e7-8116-c412f533dba1,96f173d1-1e6a-11eb-ac1d-c412f533dba1
1493,74456a8d-06de-11eb-ac18-c412f533dba1,2020-10-05 11:43:28,utm_medium,utm_source,9a87c608-52dc-11ea-abeb-c412f533dba1,00000000-0000-0000-0000-000000000000
2790,d88d5843-1d1a-11eb-ac1d-c412f533dba1,2020-11-02 18:51:11,,,9a87c608-52dc-11ea-abeb-c412f533dba1,b0a85f5c-1390-11eb-ac1d-c412f533dba1
558,2a8485fb-0b21-11eb-ac1a-c412f533dba1,2020-10-10 21:51:04,,,43756fa4-57a0-11e9-ab9a-c412f533dba1,00000000-0000-0000-0000-000000000000
2888,df8f3bba-22c0-11eb-ac1d-c412f533dba1,2020-11-09 23:22:15,,,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000
3144,f17a239d-2e0d-11eb-ac1f-c412f533dba1,2020-11-24 08:31:39,,,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000
868,45a59480-12c8-11eb-ac1c-c412f533dba1,2020-10-20 15:34:58,sms,viber,9a87c608-52dc-11ea-abeb-c412f533dba1,67964f73-d5af-11e9-abbe-c412f533dba1
1707,850458d1-1d90-11eb-ac1d-c412f533dba1,2020-11-03 08:53:31,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,e20f3e8e-1d96-11eb-ac1d-c412f533dba1
2527,c46157db-21be-11eb-ac1d-c412f533dba1,2020-11-08 16:34:43,,,b55a233c-5df6-11ea-abf2-c412f533dba1,78ef5bfe-21c2-11eb-ac1d-c412f533dba1
2089,a2d68d00-30d3-11eb-ac1f-c412f533dba1,2020-11-27 21:11:50,utm_medium,vk,db06b8bc-bfb2-11e7-8116-c412f533dba1,b061454c-6fd7-11e7-80fc-c412f533dba1


In [22]:
leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3337 entries, 0 to 3336
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   lead_id       3337 non-null   object
 1   created_at    3337 non-null   object
 2   d_utm_medium  3337 non-null   object
 3   d_utm_source  3337 non-null   object
 4   l_manager_id  3337 non-null   object
 5   l_client_id   3337 non-null   object
dtypes: object(6)
memory usage: 156.5+ KB


In [23]:
leads['created_at'] = pd.to_datetime(leads['created_at'])

In [24]:
leads.duplicated().sum()

0

In [25]:
leads.describe(include = 'all')

<ipython-input-25-9bde7c60835e>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  leads.describe(include = 'all')


,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id
count,3337,3337,3337,3337,3337,3337
unique,3337,3289,11,13,33,2183
top,270de5db-2751-11eb-ac1d-c412f533dba1,2020-10-19 12:11:53,cpc,,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000
freq,1,5,1252,1243,491,824
first,NaN,2020-10-01 00:02:55,NaN,NaN,NaN,NaN
last,NaN,2020-11-28 14:51:36,NaN,NaN,NaN,NaN


посмотрим на значения в столбце d_utm_source:

In [26]:
leads['d_utm_source'].unique()

array(['viber', 'instagram', '', 'vk', 'sms', 'google', 'utm_source',
       'insta', 'vkontakte', 'yandex', 'ycard',
       'ycard#!/tproduct/225696739-1498486363994', 'ig'], dtype=object)

Нужно объеденить:
1. 'viber', 
2. 'instagram', 'insta'
3. 'vkontakte', 'vk', 
4. 'sms', 
5. 'google', 
6. 'utm_source',
7. 'yandex', 
9. 'ycard', 'ycard#!/tproduct/225696739-1498486363994', 
10. 'ig'
11. '' - unknown

In [27]:
leads['d_utm_source'] = leads['d_utm_source'].replace('vk', 'vkontakte')
leads['d_utm_source'] = leads['d_utm_source'].replace('insta', 'instagram')
leads['d_utm_source'] = leads['d_utm_source'].replace('ycard#!/tproduct/225696739-1498486363994', 'ycard')
leads['d_utm_source'] = leads['d_utm_source'].replace('', 'unknown')

# Расчет

Соберем основной датафрейм на основе leads. Начнем с выбора временных границ:
Пока просто зададим границы дат, в последующем будем брать их командной строки при запуске пайплайн.

In [28]:
start_dt = '2020-10-15'
end_dt = '2020-11-14'

In [29]:
df = leads.query('@start_dt <= created_at <= @end_dt')

Для определения "новизны" клиента добавим в него для каждого client_id самые ранние даты лидов и транзакций из полных датафреймов trans и leads, дальше сравнивая их с start_dt заполним столбец 'new_client_id' - если есть даты меньше start_dt значит клиент "старый", если нет, то значит он появился в отчетный период "start_dt - end_dt" и будет "новым".

In [30]:
min_trans_dt = trans.groupby('l_client_id').agg({'created_at':'min'})\
                                            .reset_index()
min_trans_dt.columns = ['l_client_id','min_trans_dt']

In [31]:
min_lead_dt = leads.groupby('l_client_id').agg({'created_at':'min'}).reset_index()
min_lead_dt.columns = ['l_client_id','min_lead_dt']

In [32]:
df = df.merge(min_trans_dt, how='left')\
        .merge(min_lead_dt, how='left')

In [33]:
df['new_lead_num'] = 0
df['new_lead_num'] = df['new_lead_num'].where(df['min_lead_dt'] < start_dt, 1)

Добавим "мусорный" признак

In [34]:
df['bed_lead_num'] = 0
df['bed_lead_num'] = df['bed_lead_num'].where(df['l_client_id'] != '00000000-0000-0000-0000-000000000000', 1)

In [35]:
df

,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id,min_trans_dt,min_lead_dt,new_lead_num,bed_lead_num
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,2020-10-22 16:42:29,2020-10-16 18:08:09,1,0
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1,NaT,2020-10-15 23:45:40,1,0
2,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vkontakte,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,2020-04-17 14:48:51,2020-10-22 15:52:04,1,0
3,00aea163-13a5-11eb-ac1d-c412f533dba1,2020-10-21 17:54:59,,unknown,db06b8bc-bfb2-11e7-8116-c412f533dba1,00aea164-13a5-11eb-ac1d-c412f533dba1,NaT,2020-10-21 17:54:59,1,0
4,00b288c8-1f2a-11eb-ac1d-c412f533dba1,2020-11-05 09:44:43,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,9c1b0cfa-fd6b-11ea-ac18-c412f533dba1,NaT,2020-11-05 09:44:43,1,0
...,...,...,...,...,...,...,...,...,...,...
1896,ff7d0045-12c7-11eb-ac1c-c412f533dba1,2020-10-20 15:32:56,sms,viber,9a87c608-52dc-11ea-abeb-c412f533dba1,2e4fbe67-12cb-11eb-ac1c-c412f533dba1,NaT,2020-10-20 15:32:56,1,0
1897,ffa2180e-1f72-11eb-ac1d-c412f533dba1,2020-11-05 18:27:15,,unknown,b832aac3-fcda-11ea-ac18-c412f533dba1,ffa2180f-1f72-11eb-ac1d-c412f533dba1,2020-11-05 15:29:21,2020-11-05 18:27:15,1,0
1898,ffaa0815-16c4-11eb-ac1d-c412f533dba1,2020-10-25 17:21:33,,unknown,5dd88f32-6501-11ea-abf3-c412f533dba1,ffaa0816-16c4-11eb-ac1d-c412f533dba1,NaT,2020-10-25 17:21:33,1,0
1899,ffacb5af-194f-11eb-ac1d-c412f533dba1,2020-10-28 23:01:35,cpc,vkontakte,ad52c7a8-a752-11e7-8115-c412f533dba1,377ea5e0-d089-11e9-abbd-c412f533dba1,NaT,2020-10-28 23:01:35,1,0


добавим инфу по клубам и менеджерам:

In [36]:
df = df.merge(man, how='left', left_on = 'l_manager_id', right_on = 'manager_id')

Итоговый датафрейм должен позволять создать пивотник в гугл-шитс, сделаем предварительный вариант по первым трем метрикам церез групп бай


In [37]:
df.groupby('d_utm_source').agg({'lead_id':'count', 'bed_lead_num':'sum', 'new_lead_num':'sum'})\
                            .sort_values(by='lead_id', ascending=False)

,lead_id,bed_lead_num,new_lead_num
d_utm_source,,,
unknown,785,338,446
instagram,590,62,506
utm_source,153,12,141
viber,144,16,120
vkontakte,80,8,70
yandex,71,5,63
google,67,12,51
ycard,6,0,6
sms,5,0,5


In [38]:
df.groupby(['d_club', 'd_manager']).agg({'lead_id':'count', 'bed_lead_num':'sum', 'new_lead_num':'sum'})

lead_id  bed_lead_num  new_lead_num
d_club  d_manager                                       
club #1 manager #1       168            34           131
        manager #7       142            21           121
club #2 manager #10      105             1           104
        manager #4       166             1           163
        manager #8       135             6           124
club #3 manager #12       13             1            12
        manager #13      185             1           182
        manager #9       198             5           185
club #4 manager #11      179            64           103
        manager #2       122            13           105
        manager #5       162            25           133

Данных для пивота будет достаточно... бежим дальше...  
**кто купил в течение недели после заявки?**

Нам необходимо сопаставить транзакции и заявки...   
Прямой связи нет, поэтому будем считать, что транзакции произошедшие после создания текущей заявки и до момента следующей заявки будут относиться к этой текущей заявке (их может быть и несколько)

Определим есть ли у заявки последующая заявка, если "да" зафиксируем её номер, если нет поставим дату из "будущего"

In [39]:
df = df.sort_values(by = ['l_client_id','created_at'])

In [40]:
df['fwd_lead_dt'] = df.groupby('l_client_id')['created_at'].\
                                apply(lambda x: x.shift(-1)).\
                                fillna('2099-01-01 00:00:00')


In [41]:
df['fwd_lead_dt'] = pd.to_datetime(df['fwd_lead_dt'])

Вспомнив добрым словом SQL, возьмемся за pandas...
План: приджойнить к каждой заявке все транзакции по l_client_id, потом секвистировать все под наши условия, рассчитаем разницу между датой заявки и транзакции далее для каждой lead_id оставим l_client_id, сумму транзакций. В tmp1 - реализуем условие заявка - транзакция менее равно 7 дней, в tmp2 добавим условие "новый клиент"

In [42]:
tmp = df.merge(trans, how='left', on = 'l_client_id')

In [43]:
tmp = tmp[(tmp['created_at_y']>=tmp['created_at_x']) & (tmp['created_at_y']<=tmp['fwd_lead_dt'])]

In [44]:
tmp['days_lead_trans'] = tmp['created_at_y'] - tmp['created_at_x']

In [45]:
tmp1 = tmp[tmp['days_lead_trans'] <= '7days']

In [46]:
tmp1 = tmp1.groupby(['lead_id', 'l_client_id']).agg({'transaction_id':'nunique'}).reset_index()

In [47]:
tmp2 = tmp[(tmp['days_lead_trans'] <= '7days')&(tmp['new_lead_num'] > 0)]

In [48]:

tmp2 = tmp2.groupby(['lead_id', 'l_client_id']).agg({'m_real_amount':'sum'}).reset_index()

Теперь вернем это в основной датафрейм:

In [49]:
df = df.merge(tmp1, how='left', on = 'lead_id')

In [50]:
df = df.merge(tmp2, how='left', on = 'lead_id')

Для удобства удалим и переименуем столбцы нашего датафрейма:

In [51]:
df = df[['lead_id', 'created_at', 'd_utm_source', 'l_client_id_x', 'new_lead_num', 'bed_lead_num', 'd_manager', 'd_club', 'l_client_id_y', 'l_client_id', 'm_real_amount']]

In [52]:
df.columns = ['lead_id', 'dt', 'source', 'client_id', 'new_lead_num', 'bed_lead_num', 'manager', 'club', '7days_client_id', 'new_7days_client_id', 'new_7days_real_amount']

Проверим отсутствие NAN в категориях:

In [53]:
df['manager'].unique()

array(['manager #13', nan, 'manager #1', 'manager #5', 'manager #7',
       'manager #9', 'manager #11', 'manager #2', 'manager #10',
       'manager #8', 'manager #4', 'manager #12'], dtype=object)

In [54]:
df['manager'] = df['manager'].fillna('unknown')

In [55]:
df['club'].unique()

array(['club #3', nan, 'club #1', 'club #4', 'club #2'], dtype=object)

In [56]:
df['club'] = df['club'].fillna('unknown')

In [57]:
df['source'].unique()

array(['google', 'utm_source', 'viber', 'instagram', 'unknown', 'yandex',
       'vkontakte', 'sms', 'ycard'], dtype=object)

In [58]:
df['source'] = df['source'].fillna('unknown')

Посмотрим на результат

In [59]:
df

,lead_id,dt,source,client_id,new_lead_num,bed_lead_num,manager,club,7days_client_id,new_7days_client_id,new_7days_real_amount
0,69818bba-0ead-11eb-ac1a-c412f533dba1,2020-10-15 10:12:34,google,00000000-0000-0000-0000-000000000000,0,1,manager #13,club #3,NaN,NaN,NaN
1,a6f57c47-0eb5-11eb-ac1a-c412f533dba1,2020-10-15 11:11:32,utm_source,00000000-0000-0000-0000-000000000000,0,1,unknown,unknown,NaN,NaN,NaN
2,05a6b11b-0eb7-11eb-ac1a-c412f533dba1,2020-10-15 11:21:21,viber,00000000-0000-0000-0000-000000000000,0,1,unknown,unknown,NaN,NaN,NaN
3,47a73a00-0f05-11eb-ac1a-c412f533dba1,2020-10-15 20:41:32,instagram,00000000-0000-0000-0000-000000000000,0,1,manager #1,club #1,NaN,NaN,NaN
4,e0dcdb43-0f08-11eb-ac1a-c412f533dba1,2020-10-15 21:07:18,google,00000000-0000-0000-0000-000000000000,0,1,manager #1,club #1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1896,a904f67d-181c-11eb-ac1d-c412f533dba1,2020-10-27 10:21:35,unknown,feb21568-d5a8-11e9-abbe-c412f533dba1,1,0,manager #9,club #3,NaN,NaN,NaN
1897,ffa2180e-1f72-11eb-ac1d-c412f533dba1,2020-11-05 18:27:15,unknown,ffa2180f-1f72-11eb-ac1d-c412f533dba1,1,0,manager #13,club #3,NaN,NaN,NaN
1898,ffaa0815-16c4-11eb-ac1d-c412f533dba1,2020-10-25 17:21:33,unknown,ffaa0816-16c4-11eb-ac1d-c412f533dba1,1,0,manager #10,club #2,NaN,NaN,NaN
1899,0daaf21a-1514-11eb-ac1d-c412f533dba1,2020-10-23 13:42:24,yandex,ffaccc44-6d9e-11ea-abf3-c412f533dba1,1,0,manager #11,club #4,NaN,NaN,NaN


Проверим достаточность данных для определения нужных метрик:

In [60]:
df.groupby(['club', 'manager'])\
    .agg({'lead_id':'count', 'bed_lead_num':'sum', 'new_lead_num':'sum', 
          '7days_client_id':'nunique', 'new_7days_client_id':'nunique',
          'new_7days_real_amount':'sum'})

lead_id  bed_lead_num  new_lead_num  7days_client_id  \
club    manager                                                             
club #1 manager #1       168            34           131               29   
        manager #7       142            21           121               24   
club #2 manager #10      105             1           104               28   
        manager #4       166             1           163               31   
        manager #8       135             6           124               29   
club #3 manager #12       13             1            12                4   
        manager #13      185             1           182               23   
        manager #9       198             5           185               18   
club #4 manager #11      179            64           103               26   
        manager #2       122            13           105               25   
        manager #5       162            25           133               26   
unknown unknown          326           281            45                5   

                     new_7days_client_id  new_7days_real_amount  
club    manager                                                  
club #1 manager #1                    29               787162.0  
        manager #7                    24               443186.0  
club #2 manager #10                   28               937900.0  
        manager #4                    31               897399.0  
        manager #8                    27               564308.0  
club #3 manager #12                    4               106654.0  
        manager #13                   22               533711.0  
        manager #9                    15               590731.0  
club #4 manager #11                   26               512175.0  
        manager #2                    24               581122.0  
        manager #5                    25               496350.0  
unknown unknown                        5               142535.0

In [61]:
df.groupby('source')\
    .agg({'lead_id':'count', 'bed_lead_num':'sum', 'new_lead_num':'sum', 
          '7days_client_id':'nunique', 'new_7days_client_id':'nunique',
          'new_7days_real_amount':'sum'})\
    .sort_values(by='new_7days_real_amount', ascending=False)

,lead_id,bed_lead_num,new_lead_num,7days_client_id,new_7days_client_id,new_7days_real_amount
source,,,,,,
unknown,785,338,446,70,70,1957197.0
instagram,590,62,506,99,94,1395908.0
utm_source,153,12,141,30,30,1126387.0
viber,144,16,120,24,22,672824.0
vkontakte,80,8,70,20,20,607755.0
yandex,71,5,63,13,12,575357.0
google,67,12,51,9,9,178984.0
sms,5,0,5,1,1,75151.0
ycard,6,0,6,1,1,3670.0


Все "Ок". Экспортируем данные в таблицу test_dor и там уже согласно задания соберем пивотники.
(https://docs.google.com/spreadsheets/d/13gtDTG_NLRwTtGXNhlJk0Gn-xx1MmuSpu8RjEFVizpA/edit?usp=sharing)

In [62]:
sh = gc.open_by_key('13gtDTG_NLRwTtGXNhlJk0Gn-xx1MmuSpu8RjEFVizpA')
worksheet = sh.get_worksheet(1)

In [63]:
sh.values_clear("'Лист1'!A2:M100000") #Очистим диапозон

{'spreadsheetId': '13gtDTG_NLRwTtGXNhlJk0Gn-xx1MmuSpu8RjEFVizpA',
 'clearedRange': "'Лист1'!A2:M100000"}

In [64]:
set_with_dataframe(worksheet, df)

**Создадим пайплайн**, который будет импортировать данные из исходного гугл-шитс, предобрабатывать их и экспортировать в другую книгу гугл-шитс с уже готовым пивотником.
Необходимый диапозон дат будем указывать в командной строке при запуске скрипта:   
`python test.py --start_dt='2020-10-01' --end_dt='2020-10-05'`

test.py - https://drive.google.com/file/d/1K-SZEPgoke8N3tmVBWrJbgz7Rz54DhcL/view?usp=sharing

Дополнительно создадим дашборд Tableau для среза по источникам: https://public.tableau.com/app/profile/den3d/viz/Test202105/Dashboard2

(к сожалению в бесплатной версии не получилось сделать автообновление при изменении нашего гугл-шита, только через запуск Tableau на десктопе)